In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor as MOR
from sklearn.metrics import r2_score
from joblib import dump

In [2]:
df = pd.read_excel(r"E:\Projects\UrbanHub\files\NewDataset.xlsx")
df

,ID,Green_space_ratio,Rotation,Main_street,Sub_street,Bldg_Footprint,Type,Bldg_Centroids_x,Bldg_Centroids_y,Lengths,...,Gas_Hot,Annual_Solar_Hours,Roof_radiation_Cold,Roof_radiation_Hot,SVF,Ave_UTCI_Cold,Ave_UTCI_Hot,Ave_Percent_of_Shaded_area,Total_EUI_Cold,Total_EUI_Hot
0,B0,0.2,0,20,10,0.6,1,20.000000,30.666667,22.000000,...,0.9,448638,1158.21,3071.83,82.19,-0.47,37.07,9,8.653,8.695
1,B1,0.2,0,20,10,0.6,1,10.000000,30.666667,22.000000,...,0.6,299175,1034.74,2696.77,82.19,-0.47,37.07,9,8.653,8.695
2,B2,0.2,0,20,10,0.6,1,20.000000,77.333333,22.000000,...,0.6,264846,1042.35,2689.40,78.23,-0.47,37.07,9,8.653,8.695
3,B3,0.2,0,20,10,0.6,1,10.000000,77.333333,22.000000,...,0.9,431566,1153.40,3056.60,78.23,-0.47,37.07,9,8.653,8.695
4,B4,0.2,0,20,10,0.6,1,20.000000,124.000000,22.000000,...,0.6,265691,1060.30,2746.95,76.76,-0.47,37.07,9,8.653,8.695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,P0,0.2,90,25,5,0.6,0,-13.333333,103.750000,18.333333,...,0.0,0,0.00,0.00,84.17,0.01,36.59,16,8.779,10.197
15892,P1,0.2,90,25,5,0.6,0,33.333333,103.750000,18.333333,...,0.0,0,0.00,0.00,70.53,0.01,36.59,16,8.779,10.197
15893,P2,0.2,90,25,5,0.6,0,10.000000,58.750000,18.333333,...,0.0,0,0.00,0.00,62.54,0.01,36.59,16,8.779,10.197
15894,P3,0.2,90,25,5,0.6,0,33.333333,36.250000,18.333333,...,0.0,0,0.00,0.00,73.99,0.01,36.59,16,8.779,10.197


In [3]:
X = df[["Green_space_ratio", "Rotation", "Main_street", "Sub_street", "Bldg_Footprint", "Type", "Bldg_Centroids_x", "Bldg_Centroids_y",
        "Lengths", "Widths", "Stories", "Typology", "X", "Y", "Density", "Visibility"]]

y = df[["Cooling_Cold", "Heating_Cold", "Lighting_Cold", "Hot_water_Cold", "Gas_Cold", "Cooling_Hot", "Heating_Hot", "Lighting_Hot",
        "Hot_water_Hot", "Gas_Hot", "Annual_Solar_Hours", "Roof_radiation_Cold", "Roof_radiation_Hot", "Ave_UTCI_Cold",
        "Ave_UTCI_Hot", "Ave_Percent_of_Shaded_area", "Total_EUI_Cold", "Total_EUI_Hot"]]

In [4]:
X

,Green_space_ratio,Rotation,Main_street,Sub_street,Bldg_Footprint,Type,Bldg_Centroids_x,Bldg_Centroids_y,Lengths,Widths,Stories,Typology,X,Y,Density,Visibility
0,0.2,0,20,10,0.6,1,20.000000,30.666667,22.000000,10.000000,3,0,90,140,2.5,0.00
1,0.2,0,20,10,0.6,1,10.000000,30.666667,22.000000,10.000000,2,0,90,140,2.5,0.00
2,0.2,0,20,10,0.6,1,20.000000,77.333333,22.000000,10.000000,2,0,90,140,2.5,0.00
3,0.2,0,20,10,0.6,1,10.000000,77.333333,22.000000,10.000000,3,0,90,140,2.5,0.00
4,0.2,0,20,10,0.6,1,20.000000,124.000000,22.000000,10.000000,2,0,90,140,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,0.2,90,25,5,0.6,0,-13.333333,103.750000,18.333333,18.333333,0,0,90,140,6.5,3.29
15892,0.2,90,25,5,0.6,0,33.333333,103.750000,18.333333,18.333333,0,0,90,140,6.5,5.12
15893,0.2,90,25,5,0.6,0,10.000000,58.750000,18.333333,18.333333,0,0,90,140,6.5,5.01
15894,0.2,90,25,5,0.6,0,33.333333,36.250000,18.333333,18.333333,0,0,90,140,6.5,4.44


In [5]:
y

,Cooling_Cold,Heating_Cold,Lighting_Cold,Hot_water_Cold,Gas_Cold,Cooling_Hot,Heating_Hot,Lighting_Hot,Hot_water_Hot,Gas_Hot,Annual_Solar_Hours,Roof_radiation_Cold,Roof_radiation_Hot,Ave_UTCI_Cold,Ave_UTCI_Hot,Ave_Percent_of_Shaded_area,Total_EUI_Cold,Total_EUI_Hot
0,4827.78164,4844.44832,2588.89096,0,130.55566,3100.00248,3116.66916,3.62,0.40,0.9,448638,1158.21,3071.83,-0.47,37.07,9,8.653,8.695
1,3236.11370,3247.22482,1575.00126,0,86.11118,2083.33500,2094.44612,2.75,0.46,0.6,299175,1034.74,2696.77,-0.47,37.07,9,8.653,8.695
2,3236.11370,3247.22482,1522.22344,0,86.11118,2083.33500,2097.22390,2.75,0.46,0.6,264846,1042.35,2689.40,-0.47,37.07,9,8.653,8.695
3,4833.33720,4850.00388,2544.44648,0,130.55566,3102.78026,3119.44694,3.62,0.40,0.9,431566,1153.40,3056.60,-0.47,37.07,9,8.653,8.695
4,3236.11370,3247.22482,1572.22348,0,86.11118,2083.33500,2094.44612,2.75,0.46,0.6,265691,1060.30,2746.95,-0.47,37.07,9,8.653,8.695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00,0.01,36.59,16,8.779,10.197
15892,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00,0.01,36.59,16,8.779,10.197
15893,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00,0.01,36.59,16,8.779,10.197
15894,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00,0.01,36.59,16,8.779,10.197


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
model = MOR(XGBRegressor(n_estimators=300,learning_rate=0.1,max_depth=6))
model.fit(X_train, y_train)
[...]

[Ellipsis]

In [8]:
yp_train = model.predict(X_train)
yp_test = model.predict(X_test)

In [13]:
r_pr = [[0.2, 0, 20, 10, 0.6, 1, 20, 30.666667, 22, 10, 3, 0, 90, 140, 2.5, 0]]
pr_df = pd.DataFrame(r_pr, columns=model.feature_names_in_)
pr_res_df = pd.DataFrame(model.predict(pr_df), columns=y.columns).iloc[0,:]
results_pr = {'Real': y_test.iloc[0,:], 'Predicted': pr_res_df}
results_pr_df = pd.DataFrame(results_pr)
results_pr_df
[...]

[Ellipsis]

In [10]:
r2_train=r2_score(y_train, yp_train, multioutput='raw_values')
r2_test=r2_score(y_test, yp_test, multioutput='raw_values')

In [11]:
results = {'Target': y.columns , 'Test Score': r2_test, 'Train Score': r2_train}
results_df = pd.DataFrame(results)
results_df

,Target,Test Score,Train Score
0,Cooling_Cold,0.998185,0.999449
1,Heating_Cold,0.998121,0.999500
2,Lighting_Cold,0.990277,0.996027
3,Hot_water_Cold,0.000000,0.000000
4,Gas_Cold,0.999652,0.999951
5,Cooling_Hot,0.999441,0.999932
6,Heating_Hot,0.999656,0.999956
7,Lighting_Hot,1.000000,1.000000
8,Hot_water_Hot,0.999998,0.999998
9,Gas_Hot,1.000000,1.000000


In [12]:
dump(model, 'XGB_model.joblib')

['XGB_model.joblib']